In [2]:
from elasticsearch import Elasticsearch
es = Elasticsearch(['localhost', 'otherhost'],
    http_auth=('elastic', 'changeme'),
    port=9200
)

In [3]:
inputFile = "/home/foodmap/food-tweets-2017-04-23.json.gz"

In [4]:
import argparse
import os
import sys
import json
os.chdir("/home/foodmap/food101/")
sys.path.append(os.getcwd())
import requests
from elasticsearch import Elasticsearch
from processing.preprocess_tweet import process_tweet
from processing.twitter.Tweet import Tweet

All cities with all name:  22309
All cities unique geonameid:   23516
All countries with all names:  24782
All countries unique geonameid:   251
Len of country code dict 251


In [5]:
tweetsAsDict = Tweet.getTweetAsDictionary(inputFile)
i = 0
numIndex = 0
for tweet in tweetsAsDict:
    
    i += 1
    if i % 10000 == 0:
        print "Processed tweets: ", i
        print "Indexed tweets: ", numIndex

    new_tweet = process_tweet(tweet, forStream=False)
    if new_tweet is None:
        continue
    # new_tweet_id = new_tweet["id"]
#     print tweet
#     print "~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
#     print new_tweet
    
    # check len of img_categ
    if new_tweet["img_categories"] is not None and len(new_tweet["img_categories"]) > 0:
        new_tweet["img_category"] = new_tweet["img_categories"][0]["label"]
        new_tweet["img_category_score"] = new_tweet["img_categories"][0]["score"]
    else:
        new_tweet["img_category"] = None
        new_tweet["img_category_score"] = 0.0

    # check len of text_categ
    if len(new_tweet["text_categories"]) != 0:
        print new_tweet["text_categories"]
        for cat in new_tweet["text_categories"]:
            new_tweet["text_category"] = cat
            print "Has text categories" 
            print "Id: ", numIndex
            print "--------------------------"

            # split index per month
            es.index(index="trends04", doc_type='tweet', id=numIndex, body=new_tweet)
            numIndex += 1
    else:
        new_tweet["text_category"] = None
        # split index per month
        print "Doesn't have text categories" 
        print "Id: ", numIndex
        print "--------------------------"
        es.index(index="trends04", doc_type='tweet', id=numIndex, body=new_tweet)
        numIndex += 1

Opening file:  /home/foodmap/food-tweets-2017-04-23.json.gz
[u' pizza']
Has text categories
Id:  0
--------------------------
[u' sushi']
Has text categories
Id:  1
--------------------------
[u' cup cakes']
Has text categories
Id:  2
--------------------------
[u' tacos']
Has text categories
Id:  3
--------------------------
Doesn't have text categories
Id:  4
--------------------------
[u' ice cream']
Has text categories
Id:  5
--------------------------
Doesn't have text categories
Id:  6
--------------------------
[u' sushi']
Has text categories
Id:  7
--------------------------
Doesn't have text categories
Id:  8
--------------------------
[u' sushi', u' pho']
Has text categories
Id:  9
--------------------------
Has text categories
Id:  10
--------------------------
Doesn't have text categories
Id:  11
--------------------------
Doesn't have text categories
Id:  12
--------------------------
Doesn't have text categories
Id:  13
--------------------------
[u' carrot cake']
Has tex

KeyboardInterrupt: 

In [7]:
es.delete_by_query(index='trends04', doc_type='tweet', body={
    'query': {
        "match_all" : {}
    }
})

{u'batches': 3,
 u'deleted': 2094,
 u'failures': [],
 u'noops': 0,
 u'requests_per_second': -1.0,
 u'retries': {u'bulk': 0, u'search': 0},
 u'throttled_millis': 0,
 u'throttled_until_millis': 0,
 u'timed_out': False,
 u'took': 51,
 u'total': 2094,
 u'version_conflicts': 0}

In [6]:
es.count(index='trends04', doc_type='tweet')

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5}, u'count': 2094}

In [ ]:
# Search tweet text for QUERY
es.search(index="trends", body={"query": {"match": {'tweet_body.text':'lasagna'}}})

In [ ]:
# Search tweet for ID
es.get(index='trends', doc_type='tweet', id=855919564705673216)
# http://foodmap.isti.cnr.it:9200/trends/tweet/856023708892360704

In [ ]:
# dealing with the index 
r = requests.get('http://foodmap.isti.cnr.it:9200/',  auth=('elastic', 'changeme'))